In [1]:
import pickle
from collections import defaultdict
import pandas as pd

In [2]:
with open('./feedback.pkl', 'rb') as f:
    feedback = pickle.load(f)

In [3]:
print(type(feedback))
relevant_fields = [
    'TargetQuaternion',
    'ActualQuaternion',
    'QTarget',
    'QActual',
    'QDTarget',
    'QDActual',
    'CurrentCommandId',
    'RobotMode',
]
for x in relevant_fields:
    print(x, feedback[x][0])

<class 'numpy.ndarray'>
TargetQuaternion [ 5.55111512e-17  5.55111512e-17  7.07106769e-01 -7.07106769e-01]
ActualQuaternion [ 4.48877017e-06  2.11853167e-05  7.07106769e-01 -7.07106769e-01]
QTarget [0. 0. 0. 0. 0. 0.]
QActual [ 2.03954114e-05  0.00000000e+00  2.03954114e-05  1.37329102e-03
 -1.37329102e-03  6.86645508e-04]
QDTarget [0. 0. 0. 0. 0. 0.]
QDActual [ 0.02730266 -0.02548747 -0.01919208  1.82304382 -0.79925537  3.13316345]
CurrentCommandId 4
RobotMode 5


read feedback data

In [4]:
def read_buffered_pickle(filename='./feedback_all.pkl'):
    all_data = []
    try:
        with open(filename, 'rb') as f:
            while True:
                batch = pickle.load(f)
                all_data.extend(batch)
    except EOFError:
        pass
    return all_data

feedback = read_buffered_pickle()
print(f"Total data points loaded: {len(feedback)}")

Total data points loaded: 35600


read (command id, path) pairs

In [5]:
with open('./command_path_mapping.pkl', 'rb') as f:
    command_path_mapping = pickle.load(f)

pandas

In [6]:
df = pd.DataFrame(feedback)

data wrangling

In [7]:
df['path_id'] = df['CurrentCommandId'].map(command_path_mapping)
df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
df_sorted = df.sort_values(by=['path_id', 'TimeStamp'], ascending=True)

In [8]:
print(df_sorted)
df_sorted.to_csv('./feedback.csv')

                                        TargetQuaternion  \
90     [-1.1379786002407855e-15, -1.0, -3.33066907387...   
91     [-1.3322676295501878e-15, -1.0, -2.77555756156...   
92     [-1.2212453270876722e-15, -1.0, -2.49800180540...   
93     [-1.2490009027033011e-15, -1.0, -2.22044604925...   
94     [-1.1934897514720433e-15, -1.0, -2.77555756156...   
...                                                  ...   
35595  [-9.43689570931383e-16, -0.9999978542327881, -...   
35596  [-9.43689570931383e-16, -0.9999980330467224, -...   
35597  [-9.159339953157541e-16, -0.999998152256012, -...   
35598  [-5.828670879282072e-16, -0.9999983310699463, ...   
35599  [-7.771561172376096e-16, -0.9999985098838806, ...   

                                        ActualQuaternion  \
90     [-2.9509017622331157e-05, -1.0, -3.32235049427...   
91     [1.0421446177133475e-06, -1.0, -3.322359361845...   
92     [-3.752306383830728e-06, -1.0, -9.433197192265...   
93     [-1.879096271295566e-05, -1.0, 2